# Loading the dataset

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dilwong/flightprices")

print("Path to dataset files:", path)

ModuleNotFoundError: No module named 'kagglehub'

In [3]:
import pandas as pd

df = pd.read_csv("data/itineraries.csv")
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,0,False,False,...,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,0,False,False,...,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,0,False,False,...,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach


# Pre-processing

## List out the most popular routes

In [4]:
from collections import Counter

route_counts = Counter()

for chunk in pd.read_csv("data/itineraries.csv", usecols=["startingAirport", "destinationAirport"], chunksize=500_000):
    chunk = chunk.dropna(subset=["startingAirport", "destinationAirport"])

    routes = chunk["startingAirport"] + "-" + chunk["destinationAirport"]

    route_counts.update(routes)

route_df = pd.DataFrame(route_counts.most_common(20), columns=["Route", "Count"])
print(route_df)

      Route   Count
0   ATL-LAX  709809
1   LAX-BOS  679169
2   LGA-LAX  677713
3   LAX-ATL  669609
4   LAX-LGA  663659
5   BOS-LAX  644390
6   LAX-JFK  625496
7   LAX-ORD  620576
8   DFW-LAX  612390
9   LAX-DFW  610669
10  JFK-LAX  605017
11  LAX-DTW  601537
12  ORD-LAX  597847
13  LAX-EWR  587270
14  DTW-LAX  582022
15  CLT-LAX  572097
16  JFK-ORD  557152
17  LAX-CLT  554474
18  LGA-ORD  550319
19  LAX-PHL  549880


## List out the most popular airlines on LAX - JFK

In [5]:
origin = "LAX"
destination = "JFK"

airline_counts = Counter()

for chunk in pd.read_csv("data/itineraries.csv", usecols=["startingAirport", "destinationAirport", "segmentsAirlineName"], chunksize=500_000):
    chunk = chunk.dropna(subset=["startingAirport", "destinationAirport", "segmentsAirlineName"])

    mask = (chunk["startingAirport"] == origin) & (chunk["destinationAirport"] == destination)
    airline_counts.update(chunk.loc[mask, "segmentsAirlineName"])

airline_df = pd.DataFrame(airline_counts.most_common(), columns=["Airline", "FlightCount"])
print(airline_df.head(10))

                                Airline  FlightCount
0  American Airlines||American Airlines       151007
1                       JetBlue Airways       113339
2                     American Airlines        93532
3      Alaska Airlines||Alaska Airlines        84522
4                                 Delta        71229
5                          Delta||Delta        41616
6      JetBlue Airways||JetBlue Airways        23101
7                                United        16705
8                        United||United        14394
9               United||Alaska Airlines         7268


## Only keep LAX - JFK American Airlines flights

In [6]:
origin = "LAX"
destination = "JFK"
airline = "American Airlines"

chunks = []

for chunk in pd.read_csv("data/itineraries.csv", chunksize=500_000):
    mask = (
        (chunk["startingAirport"] == origin)
        & (chunk["destinationAirport"] == destination)
        & (chunk["segmentsAirlineName"].str.contains(airline, case=False, na=False))
    )
    filtered = chunk.loc[mask].copy()
    if not filtered.empty:
        chunks.append(filtered)

df = pd.concat(chunks)
print(df.shape)

(246932, 27)


## List out missing values

In [7]:
missing = df.isnull().sum().sort_values(ascending=False)
missing_percent = (missing / len(df)) * 100

missing_summary = pd.DataFrame({
    "Missing Values": missing,
    "Percent Missing": missing_percent.round(2)
})

print(missing_summary.head(15))

                                   Missing Values  Percent Missing
totalTravelDistance                          1432             0.58
segmentsDistance                              358             0.14
legId                                           0             0.00
segmentsDurationInSeconds                       0             0.00
segmentsEquipmentDescription                    0             0.00
segmentsAirlineCode                             0             0.00
segmentsAirlineName                             0             0.00
segmentsDepartureAirportCode                    0             0.00
segmentsArrivalAirportCode                      0             0.00
segmentsArrivalTimeRaw                          0             0.00
segmentsArrivalTimeEpochSeconds                 0             0.00
segmentsDepartureTimeRaw                        0             0.00
segmentsDepartureTimeEpochSeconds               0             0.00
seatsRemaining                                  0             

## Drop rows with missing values and duplicates

In [10]:
df = df.dropna(subset=["totalTravelDistance", "segmentsDistance"])
df = df.drop_duplicates()
print(df.shape)

(245500, 27)


## Further pre-process
Since only 5% are economy tickets, drop those rows and only care about non-economy fares)

In [11]:
cols = df.columns.tolist()
df_essential = df[["searchDate", "flightDate", "totalFare", "isBasicEconomy"]]
df_non_basic_economy = df_essential[df_essential['isBasicEconomy'] == False]
df_non_basic_economy = df_non_basic_economy.drop(columns=['isBasicEconomy'])

print(df_non_basic_economy.head())
print(df_non_basic_economy.shape)

      searchDate  flightDate  totalFare
4815  2022-04-16  2022-04-17      366.6
4818  2022-04-16  2022-04-17      366.6
4820  2022-04-16  2022-04-17      366.6
4822  2022-04-16  2022-04-17      366.6
4823  2022-04-16  2022-04-17      366.6
(230737, 3)


Since the dataset starts with flights at April 17, there is no search price data earlier than that (e.g. the flight on 17th of April only has search price history for the 17th of April). Therefore, dropped all the rows with flightDate before June 1 (example: year-06-01). Since we only want to predict for summer, also dropped all the tables after August 31th)

In [18]:
df_non_basic_economy['flightDate'] = pd.to_datetime(df_non_basic_economy['flightDate'])
df_non_basic_economy_filtered = df_non_basic_economy[df_non_basic_economy['flightDate'] >= pd.to_datetime('2022-06-01')]
df = df_non_basic_economy_filtered[df_non_basic_economy_filtered['flightDate'] <= pd.to_datetime('2022-08-31')]

print(df)
print(df.shape)

          searchDate flightDate  totalFare
485631    2022-04-17 2022-06-01     272.60
485638    2022-04-17 2022-06-01     281.60
485639    2022-04-17 2022-06-01     281.60
485640    2022-04-17 2022-06-01     281.60
485650    2022-04-17 2022-06-01     328.60
...              ...        ...        ...
64430406  2022-08-30 2022-08-31     527.60
64430411  2022-08-30 2022-08-31    1412.61
64430412  2022-08-30 2022-08-31    1738.60
64430413  2022-08-30 2022-08-31    1738.60
64430414  2022-08-30 2022-08-31    1738.60

[130450 rows x 3 columns]
(130450, 3)


## Add features for training

In [33]:
df["searchDate"] = pd.to_datetime(df["searchDate"], errors="coerce")
df["flightDate"] = pd.to_datetime(df["flightDate"], errors="coerce")

df["days_to_departure"] = (df["flightDate"] - df["searchDate"]).dt.days

df["search_dow"] = df["searchDate"].dt.dayofweek
df["flight_dow"] = df["flightDate"].dt.dayofweek

df["search_month"] = df["searchDate"].dt.month
df["flight_month"] = df["flightDate"].dt.month

df["search_week"] = df["searchDate"].dt.isocalendar().week.astype(int)
df["flight_week"] = df["flightDate"].dt.isocalendar().week.astype(int)

df["is_weekend_flight"] = (
    df["flight_dow"].isin([5, 6]).astype(int)
)

summer_start = pd.to_datetime("2022-06-01")
df["days_into_summer"] = (df["flightDate"] - summer_start).dt.days

df["is_peak_travel_period"] = (
    (df["flightDate"] >= pd.to_datetime("2022-07-01"))
    & (df["flightDate"] <= pd.to_datetime("2022-07-31"))
).astype(int)

df.head()

,searchDate,flightDate,totalFare,days_to_departure,search_dow,flight_dow,search_month,flight_month,search_week,flight_week,is_weekend_flight,days_into_summer,is_peak_travel_period
485631,2022-04-17,2022-06-01,272.6,45,6,2,4,6,15,22,0,0,0
485638,2022-04-17,2022-06-01,281.6,45,6,2,4,6,15,22,0,0,0
485639,2022-04-17,2022-06-01,281.6,45,6,2,4,6,15,22,0,0,0
485640,2022-04-17,2022-06-01,281.6,45,6,2,4,6,15,22,0,0,0
485650,2022-04-17,2022-06-01,328.6,45,6,2,4,6,15,22,0,0,0


# Save the dataset

In [34]:
df.to_csv("data/preprocessed_dataset.csv", index=False)